In [2]:
import pickle
import numpy as np
from ctypes import *
from scipy.interpolate import LinearNDInterpolator, interp1d
from obspy import UTCDateTime
import obspy
from matplotlib import pyplot as plt
from scipy.ndimage import uniform_filter1d
from scipy.stats import median_abs_deviation
from obspy.signal.trigger import classic_sta_lta
import bisect
import os
import concurrent.futures
from itertools import repeat, product
from wpca import WPCA
from scipy.spatial.distance import cdist, euclidean
from skimage.transform import radon, iradon
import pygmt
import xarray
from scipy.signal import find_peaks
import SPECFEM3D_interface

In [2]:
def calc_times(source_x, source_y, source_z, vp, vs, x_m, y_m, z_m, ind):
    so_file = "/home/moshe/eyal/eikonal/Eikonal3D.so"
    eikonal_3d = CDLL(so_file)
    size_x = np.ma.size(vp, axis=0)
    size_y = np.ma.size(vp, axis=1)
    size_z = np.ma.size(vp, axis=2)
    vp_flat = np.ndarray.flatten(vp, 'C')
    vs_flat = np.ndarray.flatten(vs, 'C')
    v_dim = [size_x, size_y, size_z]
    d_elem = [x_m/(size_x-1), y_m/(size_y-1), z_m/(size_z-1)]
    source = [int(source_x/(x_m/(size_x-1))), int(source_y/(y_m/(size_y-1))), int(source_z/(z_m/(size_z-1)))]
    print(source)

    eikonal_3d_func = eikonal_3d._Z9eikonal3DPfPiS_S0_S_
    eikonal_3d_func.restype = None
    vp_flatc = (c_float * len(vp_flat))(*vp_flat)
    vs_flatc = (c_float * len(vs_flat))(*vs_flat)
    vDim = (c_int * len(v_dim))(*v_dim)
    dElem = (c_float * len(d_elem))(*d_elem)
    sourceC = (c_int * len(source))(*source)

    times_vp = (c_float * len(vp_flat))()
    eikonal_3d_func(vp_flatc, vDim, dElem, sourceC, times_vp)
    times_vp = np.array(times_vp[:])
    times_vp = times_vp.reshape((size_z, size_y, size_x))
    times_vp = times_vp.transpose((2, 1, 0))

    times_vs = (c_float * len(vs_flat))()
    eikonal_3d_func(vs_flatc, vDim, dElem, sourceC, times_vs)
    times_vs = np.array(times_vs[:])
    times_vs = times_vs.reshape((size_z, size_y, size_x))
    times_vs = times_vs.transpose((2, 1, 0))

    return ind, times_vp, times_vs

In [3]:
def get_velocity_models(size):
    tomographic_models = SPECFEM3D_interface.read_tomographic_models("test", 0)
    ind = 0
    vp = np.zeros((size + 1, size + 1, size + 1))
    vs = np.zeros((size + 1, size + 1, size + 1))
    for model in tomographic_models:
        reshaped_vp_model = np.reshape(model[:, 3], (321, 321, int(np.ma.size(model, 0)/321**2)),
                                       order='F')
        reshaped_vs_model = np.reshape(model[:, 4], (321, 321, int(np.ma.size(model, 0)/321**2)),
                                       order='F')
        if ind == 0:
            depth = int(np.ma.size(reshaped_vp_model, 2) / 4) + 1
            start_depth = 0
        else:
            depth = int(np.ma.size(reshaped_vp_model, 2) / 4)
            start_depth = 1
        vp[:, :, ind:(ind + depth)] = reshaped_vp_model[0::4, 0::4, start_depth::4]
        vs[:, :, ind:(ind + depth)] = reshaped_vs_model[0::4, 0::4, start_depth::4]
        ind += depth
    return vp, vs

In [4]:
v_p, v_s = get_velocity_models(80)

In [5]:
x_m = 20000
y_m = 20000
z_m = 20000

In [6]:
stations_points_num_1 = 9
stations_points_num_2 = 9
eikonal_xs = np.concatenate((np.tile(np.asarray([10000, 10000, 10250, 10250]), stations_points_num_1), 
                             np.tile(np.asarray([10000, 10000, 10250, 10250]), stations_points_num_2)))
eikonal_ys = np.concatenate((np.tile(np.asarray([1000, 1250]), stations_points_num_1 * 2), 
                             np.tile(np.asarray([19000, 19250]), stations_points_num_2 * 2)))
eikonal_zs = np.concatenate((np.repeat(np.linspace(0, 2000, stations_points_num_1), 4), 
                             np.repeat(np.linspace(0, 2000, stations_points_num_2), 4)))
eikonal_fibre_vs = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2), 
                             len(eikonal_zs)))
eikonal_fibre_vp = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2),
                             len(eikonal_zs)))
indices = list(range(len(eikonal_xs)))
with concurrent.futures.ProcessPoolExecutor(max_workers=72) as executor:
    for i, eikonal_fibre_vp_curr, eikonal_fibre_vs_curr in executor.map(calc_times, eikonal_xs, eikonal_ys, eikonal_zs, 
                                                                        repeat(v_p), repeat(v_s), repeat(x_m), 
                                                                        repeat(y_m), repeat(z_m), indices):
        eikonal_fibre_vp[:, :, :, i] = eikonal_fibre_vp_curr
        eikonal_fibre_vs[:, :, :, i] = eikonal_fibre_vs_curr

[40, 4, 0]

[40, 5, 0][41, 4, 0]
[41, 5, 0]
[40, 4, 1]
[40, 5, 1]
[41, 4, 1]
[41, 5, 1]
[40, 4, 2]
[40, 5, 2]
[41, 4, 2]
[41, 5, 2]
[40, 4, 3]
[40, 5, 3]
[41, 4, 3]
[41, 5, 3]
[40, 4, 4]
[40, 5, 4]
[41, 4, 4]
[41, 5, 4]
[40, 4, 5]
[40, 5, 5]
[41, 4, 5]
[41, 5, 5]
[40, 4, 6]
[40, 5, 6]
[41, 4, 6]
[41, 5, 6]
[40, 4, 7]
[40, 5, 7]
[41, 4, 7]
[41, 5, 7]
[40, 4, 8]
[40, 5, 8]
[41, 4, 8]
[41, 5, 8]
[40, 76, 0]
[40, 77, 0]
[41, 76, 0]
[41, 77, 0]
[40, 76, 1]
[40, 77, 1]
[41, 76, 1]
[41, 77, 1]
[40, 76, 2]
[40, 77, 2]
[41, 76, 2]
[41, 77, 2]
[40, 76, 3]
[40, 77, 3]
[41, 76, 3]
[41, 77, 3]
[40, 76, 4]
[40, 77, 4]
[41, 76, 4]
[41, 77, 4]
[40, 76, 5]
[40, 77, 5]
[41, 76, 5]
[41, 77, 5]
[40, 76, 6]
[40, 77, 6]
[41, 76, 6]
[41, 77, 6]
[40, 76, 7]
[40, 77, 7]
[41, 76, 7]
[41, 77, 7]
[40, 76, 8]
[40, 77, 8]
[41, 76, 8]
[41, 77, 8]


In [36]:
surface_points = product(np.linspace(0, 20000, 81).astype(int), np.linspace(0, 20000, 81).astype(int))
surface_points = np.array([*list(surface_points)])

eikonal_xs = surface_points[:, 0]
eikonal_ys = surface_points[:, 1]
eikonal_zs = np.zeros((len(eikonal_xs)))
eikonal_surface_fibre_vp = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2), 
                             len(eikonal_zs)))
eikonal_surface_fibre_vs = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2),
                             len(eikonal_zs)))
indices = list(range(len(eikonal_xs)))
with concurrent.futures.ProcessPoolExecutor(max_workers=162) as executor:
    for i, eikonal_fibre_vp_curr, eikonal_fibre_vs_curr in executor.map(calc_times, eikonal_xs, eikonal_ys, eikonal_zs, 
                                                                        repeat(v_p), repeat(v_s), repeat(x_m), 
                                                                        repeat(y_m), repeat(z_m), indices):
        eikonal_surface_fibre_vp[:, :, :, i] = eikonal_fibre_vp_curr
        eikonal_surface_fibre_vs[:, :, :, i] = eikonal_fibre_vs_curr

[0, 0, 0]
[0, 1, 0]
[0, 2, 0]
[0, 3, 0]
[0, 4, 0]
[0, 5, 0]
[0, 6, 0]
[0, 7, 0]
[0, 8, 0]
[0, 9, 0]
[0, 10, 0]
[0, 11, 0]
[0, 12, 0]
[0, 13, 0]
[0, 14, 0]
[0, 15, 0]
[0, 16, 0]
[0, 17, 0]
[0, 18, 0]
[0, 19, 0]
[0, 20, 0]
[0, 21, 0]
[0, 22, 0]
[0, 23, 0]
[0, 24, 0]
[0, 25, 0]
[0, 26, 0]
[0, 27, 0]
[0, 28, 0]
[0, 29, 0]
[0, 30, 0]
[0, 31, 0]
[0, 32, 0]
[0, 33, 0]
[0, 34, 0]
[0, 35, 0]
[0, 36, 0]
[0, 37, 0]
[0, 38, 0]
[0, 39, 0]
[0, 40, 0]
[0, 41, 0]
[0, 42, 0]
[0, 43, 0]
[0, 44, 0]
[0, 45, 0]
[0, 46, 0]
[0, 47, 0]
[0, 48, 0]
[0, 49, 0]
[0, 50, 0]
[0, 51, 0]
[0, 52, 0]
[0, 53, 0]
[0, 54, 0]
[0, 55, 0]
[0, 56, 0]
[0, 57, 0]
[0, 58, 0]
[0, 59, 0]
[0, 60, 0]
[0, 61, 0]
[0, 62, 0]
[0, 63, 0]
[0, 64, 0]
[0, 65, 0]
[0, 66, 0]
[0, 67, 0]
[0, 68, 0]
[0, 69, 0]
[0, 70, 0]
[0, 71, 0]
[0, 72, 0]
[0, 73, 0]
[0, 74, 0]
[0, 75, 0]
[0, 76, 0]
[0, 77, 0]
[0, 78, 0]
[0, 79, 0]
[0, 80, 0]
[1, 0, 0]
[1, 1, 0]
[1, 2, 0]
[1, 3, 0]
[1, 4, 0]
[1, 5, 0]
[1, 6, 0]
[1, 7, 0]
[1, 8, 0]
[1, 9, 0]
[1, 10, 0]
[1, 11, 

[17, 70, 0]
[17, 71, 0]
[17, 72, 0]
[17, 73, 0]
[17, 74, 0]
[17, 75, 0]
[17, 76, 0]
[17, 77, 0]
[17, 78, 0]
[17, 79, 0]
[17, 80, 0]
[18, 0, 0]
[18, 1, 0]
[18, 2, 0]
[18, 3, 0]
[18, 4, 0]
[18, 5, 0]
[18, 6, 0]
[18, 7, 0]
[18, 8, 0]
[18, 9, 0]
[18, 10, 0]
[18, 11, 0]
[18, 12, 0]
[18, 13, 0]
[18, 14, 0]
[18, 15, 0]
[18, 16, 0]
[18, 17, 0]
[18, 18, 0]
[18, 19, 0]
[18, 20, 0]
[18, 21, 0]
[18, 22, 0]
[18, 23, 0]
[18, 24, 0]
[18, 25, 0]
[18, 26, 0]
[18, 27, 0]
[18, 28, 0]
[18, 29, 0]
[18, 30, 0]
[18, 31, 0]
[18, 32, 0]
[18, 33, 0]
[18, 34, 0]
[18, 35, 0]
[18, 36, 0]
[18, 37, 0]
[18, 38, 0]
[18, 39, 0]
[18, 40, 0]
[18, 41, 0]
[18, 42, 0]
[18, 43, 0]
[18, 44, 0]
[18, 45, 0]
[18, 46, 0]
[18, 47, 0]
[18, 48, 0]
[18, 49, 0]
[18, 50, 0]
[18, 51, 0]
[18, 52, 0]
[18, 53, 0]
[18, 54, 0]
[18, 55, 0]
[18, 56, 0]
[18, 57, 0]
[18, 58, 0]
[18, 59, 0]
[18, 60, 0]
[18, 61, 0]
[18, 62, 0]
[18, 63, 0]
[18, 64, 0]
[18, 65, 0]
[18, 66, 0]
[18, 67, 0]
[18, 68, 0]
[18, 69, 0]
[18, 70, 0]
[18, 71, 0]
[18, 72, 0]
[1

[34, 72, 0]
[34, 73, 0]
[34, 74, 0]
[34, 75, 0]
[34, 76, 0]
[34, 77, 0]
[34, 78, 0]
[34, 79, 0]
[34, 80, 0]
[35, 0, 0]
[35, 1, 0]
[35, 2, 0]
[35, 3, 0]
[35, 4, 0]
[35, 5, 0]
[35, 6, 0]
[35, 7, 0]
[35, 8, 0]
[35, 9, 0]
[35, 10, 0]
[35, 11, 0]
[35, 12, 0]
[35, 13, 0]
[35, 14, 0]
[35, 15, 0]
[35, 16, 0]
[35, 17, 0]
[35, 18, 0]
[35, 19, 0]
[35, 20, 0]
[35, 21, 0]
[35, 22, 0]
[35, 23, 0]
[35, 24, 0]
[35, 25, 0]
[35, 26, 0]
[35, 27, 0]
[35, 28, 0]
[35, 29, 0]
[35, 30, 0]
[35, 31, 0]
[35, 32, 0]
[35, 33, 0]
[35, 34, 0]
[35, 35, 0]
[35, 36, 0]
[35, 37, 0]
[35, 38, 0]
[35, 39, 0]
[35, 40, 0]
[35, 41, 0]
[35, 42, 0]
[35, 43, 0]
[35, 44, 0]
[35, 45, 0]
[35, 46, 0]
[35, 47, 0]
[35, 48, 0]
[35, 49, 0]
[35, 50, 0]
[35, 51, 0]
[35, 52, 0]
[35, 53, 0]
[35, 54, 0]
[35, 55, 0]
[35, 56, 0]
[35, 57, 0]
[35, 58, 0]
[35, 59, 0]
[35, 60, 0]
[35, 61, 0]
[35, 62, 0]
[35, 63, 0]
[35, 64, 0]
[35, 65, 0]
[35, 66, 0]
[35, 67, 0]
[35, 68, 0]
[35, 69, 0]
[35, 70, 0]
[35, 71, 0]
[35, 72, 0]
[35, 73, 0]
[35, 74, 0]
[3

[51, 74, 0]
[51, 76, 0]
[51, 75, 0]
[51, 77, 0]
[51, 78, 0]
[51, 79, 0]
[51, 80, 0]
[52, 0, 0]
[52, 1, 0]
[52, 2, 0]
[52, 3, 0]
[52, 4, 0]
[52, 5, 0]
[52, 6, 0]
[52, 7, 0]
[52, 8, 0]
[52, 9, 0]
[52, 10, 0]
[52, 11, 0]
[52, 12, 0]
[52, 13, 0]
[52, 14, 0]
[52, 15, 0]
[52, 16, 0]
[52, 17, 0]
[52, 18, 0]
[52, 19, 0]
[52, 20, 0]
[52, 21, 0]
[52, 22, 0]
[52, 23, 0]
[52, 24, 0]
[52, 25, 0]
[52, 26, 0]
[52, 27, 0]
[52, 28, 0]
[52, 29, 0]
[52, 30, 0]
[52, 31, 0]
[52, 32, 0]
[52, 33, 0]
[52, 34, 0]
[52, 35, 0]
[52, 36, 0]
[52, 37, 0]
[52, 38, 0]
[52, 39, 0]
[52, 40, 0]
[52, 41, 0]
[52, 42, 0]
[52, 43, 0]
[52, 44, 0]
[52, 45, 0]
[52, 46, 0]
[52, 47, 0]
[52, 48, 0]
[52, 49, 0]
[52, 50, 0]
[52, 51, 0]
[52, 52, 0]
[52, 53, 0]
[52, 54, 0]
[52, 55, 0]
[52, 56, 0]
[52, 57, 0]
[52, 58, 0]
[52, 59, 0]
[52, 60, 0]
[52, 61, 0]
[52, 62, 0]
[52, 63, 0]
[52, 64, 0]
[52, 65, 0]
[52, 66, 0]
[52, 67, 0]
[52, 68, 0]
[52, 69, 0]
[52, 70, 0]
[52, 71, 0]
[52, 73, 0]
[52, 72, 0]
[52, 74, 0]
[52, 75, 0]
[52, 76, 0]
[5

[68, 76, 0]
[68, 77, 0]
[68, 78, 0]
[68, 79, 0]
[68, 80, 0]
[69, 0, 0]
[69, 1, 0]
[69, 2, 0]
[69, 3, 0]
[69, 4, 0]
[69, 5, 0]
[69, 6, 0]
[69, 7, 0]
[69, 8, 0]
[69, 9, 0]
[69, 10, 0]
[69, 11, 0]
[69, 12, 0]
[69, 13, 0]
[69, 14, 0]
[69, 15, 0]
[69, 16, 0]
[69, 17, 0]
[69, 18, 0]
[69, 19, 0]
[69, 20, 0]
[69, 21, 0]
[69, 22, 0]
[69, 23, 0]
[69, 24, 0]
[69, 25, 0]
[69, 26, 0]
[69, 27, 0]
[69, 28, 0]
[69, 29, 0]
[69, 30, 0]
[69, 31, 0]
[69, 32, 0]
[69, 33, 0]
[69, 34, 0]
[69, 35, 0]
[69, 36, 0]
[69, 37, 0]
[69, 38, 0]
[69, 39, 0]
[69, 40, 0]
[69, 41, 0]
[69, 42, 0]
[69, 43, 0]
[69, 44, 0]
[69, 45, 0]
[69, 46, 0]
[69, 47, 0]
[69, 48, 0]
[69, 49, 0]
[69, 50, 0]
[69, 51, 0]
[69, 52, 0]
[69, 53, 0]
[69, 54, 0]
[69, 55, 0]
[69, 56, 0]
[69, 57, 0]
[69, 58, 0]
[69, 59, 0]
[69, 60, 0]
[69, 61, 0]
[69, 62, 0]
[69, 63, 0]
[69, 64, 0]
[69, 65, 0]
[69, 66, 0]
[69, 67, 0]
[69, 68, 0]
[69, 69, 0]
[69, 70, 0]
[69, 71, 0]
[69, 72, 0]
[69, 73, 0]
[69, 74, 0]
[69, 75, 0]
[69, 76, 0]
[69, 77, 0]
[69, 78, 0]
[6

In [37]:
surface_points = product(np.linspace(0, 20000, 81).astype(int), np.linspace(0, 20000, 81).astype(int))
surface_points = np.array([*list(surface_points)])

In [38]:
import data

stations = data.get_stations_xyz()

In [39]:
stations_x = [station.longitude + 10000 for station in stations if 'BI' not in station.station]
stations_y = [station.latitude + 10000 for station in stations if 'BI' not in station.station]

In [40]:
stations_z = np.zeros((len(stations_y)))
stations_coord_surface = np.vstack((stations_x, stations_y, stations_z)).T

In [7]:
stations_x_1 = np.zeros((201)) + 10000
stations_y_1 = np.zeros((201)) + 1000
stations_z_1 = np.linspace(0, 2000, 201)
stations_coord_1 = np.vstack((stations_x_1, stations_y_1, stations_z_1)).T
stations_x_2 = np.zeros((201)) + 10000
stations_y_2 = np.zeros((201)) + 19000
stations_z_2 = np.linspace(0, 2000, 201)
stations_coord_2 = np.vstack((stations_x_2, stations_y_2, stations_z_2)).T
stations_coord = np.concatenate((stations_coord_1, stations_coord_2))

In [8]:
stations_eikonal_tables_vp = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2),
                                       len(stations_x_1) + len(stations_x_2)))
stations_eikonal_tables_vs = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2),
                                       len(stations_x_1) + len(stations_x_2)))
coords_xs_1 = [10000, 10250]
coords_ys_1 = [1000, 1250]
coords_xs_2 = [10000, 10250]
coords_ys_2 = [19000, 19250]
coords_1 = np.array(np.meshgrid(coords_xs_1, coords_ys_1, np.linspace(0, 2000, stations_points_num_1))).T.reshape(-1, 3)
coords_2 = np.array(np.meshgrid(coords_xs_2, coords_ys_2, np.linspace(0, 2000, stations_points_num_2))).T.reshape(-1, 3)
coords = np.concatenate((coords_1, coords_2))
for k in range(np.ma.size(v_p, axis=0)):
    for j in range(np.ma.size(v_p, axis=1)):
        for i in range(np.ma.size(v_p, axis=2)):
            vp_interpolator = LinearNDInterpolator(coords, eikonal_fibre_vp[k, j, i, :])
            stations_eikonal_tables_vp[k, j, i, :] = vp_interpolator(stations_coord)
            vs_interpolator = LinearNDInterpolator(coords, eikonal_fibre_vs[k, j, i, :])
            stations_eikonal_tables_vs[k, j, i, :] = vs_interpolator(stations_coord)

In [41]:
stations_eikonal_tables_vp_surface = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2),
                                               len(stations_x)))
stations_eikonal_tables_vs_surface = np.zeros((np.ma.size(v_p, axis=0), np.ma.size(v_p, axis=1), np.ma.size(v_p, axis=2),
                                               len(stations_x)))

coords_surface = surface_points
for k in range(np.ma.size(v_p, axis=0)):
    for j in range(np.ma.size(v_p, axis=1)):
        for i in range(np.ma.size(v_p, axis=2)):
            vp_surface_interpolator = LinearNDInterpolator(coords_surface, eikonal_surface_fibre_vp[k, j, i, :])
            stations_eikonal_tables_vp_surface[k, j, i, :] = vp_surface_interpolator(stations_coord_surface[:, :2])
            vs_surface_interpolator = LinearNDInterpolator(coords_surface, eikonal_surface_fibre_vs[k, j, i, :])
            stations_eikonal_tables_vs_surface[k, j, i, :] = vs_surface_interpolator(stations_coord_surface[:, :2])

In [42]:
stations_eikonal_tables_vp_total = np.concatenate((stations_eikonal_tables_vp, stations_eikonal_tables_vp_surface), axis=3)
stations_eikonal_tables_vs_total = np.concatenate((stations_eikonal_tables_vs, stations_eikonal_tables_vs_surface), axis=3)

In [40]:
def get_s_index(subarray_sum):
    s_sus_p_max = int(np.where(subarray_sum > subarray_sum.max()/5)[0][0] + 0.5/dt)
    if s_sus_p_max >= len(subarray_sum):
        return subarray_sum.argmax()
    s_sus_s_max = subarray_sum[s_sus_p_max:].max()
    s_sus_s_argmax = subarray_sum[s_sus_p_max:].argmax() + s_sus_p_max
    if s_sus_p_max == s_sus_s_argmax:
        return subarray_sum.argmax()
    if s_sus_s_max / subarray_sum.max() > 0.5 and subarray_sum[s_sus_p_max:s_sus_s_argmax].min() / s_sus_s_max < 0.5:
        return s_sus_s_argmax
    else:
        return subarray_sum.argmax()

In [58]:
def calc_scores_slice(waveform_array, dt, dims, stations_eikonal_tables_vp, stations_eikonal_tables_vs, i, a1_indices, a2_indices):
    score = np.zeros((dims[1], dims[2]))
    std_score = np.zeros((dims[1], dims[2]))
    mad_score = np.zeros((dims[1], dims[2]))
    
    for j in range(dims[1]):
        for k in range(dims[2]):
            shifted_waveform_array_p = np.zeros(waveform_array.shape)
            shifted_waveform_array_s = np.zeros(waveform_array.shape)
            for l in range(np.ma.size(waveform_array, axis=0)):
                shifted_waveform_array_p[l, :] = \
                    np.roll(waveform_array[l, :], -int(np.round(stations_eikonal_tables_vp[i, j, k, l] / dt)))
                shifted_waveform_array_s[l, :] = \
                    np.roll(waveform_array[l, :], -int(np.round(stations_eikonal_tables_vs[i, j, k, l] / dt)))

            shifted_waveform_array_p_a1 = shifted_waveform_array_p[a1_indices, :]
            shifted_waveform_array_p_a2 = shifted_waveform_array_p[a2_indices, :]
            shifted_waveform_array_s_a1 = shifted_waveform_array_s[a1_indices, :]
            shifted_waveform_array_s_a1 = shifted_waveform_array_s[a2_indices, :]

            p_a1_sum = shifted_waveform_array_p_a1.sum(axis=0) ** 2 / \
                       (shifted_waveform_array_p_a1 ** 2).sum(axis=0) / len(a1_indices)**2 * \
                       np.abs(shifted_waveform_array_p_a1.sum(axis=0))
                        p_a2_sum = shifted_waveform_array_p_a2.sum(axis=0) ** 2 / \
                                   (shifted_waveform_array_p_a2 ** 2).sum(axis=0) / len(a2_indices)**2 * \
                                   np.abs(shifted_waveform_array_p_a2.sum(axis=0))
                        s_a1_sum = shifted_waveform_array_s_a1.sum(axis=0) ** 2 / \
                                   (shifted_waveform_array_s_a1 ** 2).sum(axis=0) / len(a1_indices)**2 * \
                                   np.abs(shifted_waveform_array_s_a1.sum(axis=0))
                        s_a2_sum = shifted_waveform_array_s_a2.sum(axis=0) ** 2 / \
                                   (shifted_waveform_array_s_a2 ** 2).sum(axis=0) / len(a2_indices)**2 * \
                                   np.abs(shifted_waveform_array_s_a2.sum(axis=0))

            p_sum = (shifted_waveform_array_p ** 2).sum(axis=0)
            s_sum = (shifted_waveform_array_s ** 2).sum(axis=0)

            p_sum = uniform_filter1d(p_sum, 30, mode='reflect')
            s_sum = uniform_filter1d(s_sum, 30, mode='reflect')
            
            p_sum = classic_sta_lta(p_sum, int(0.1/dt), int(0.5/dt))
            s_sum = classic_sta_lta(s_sum, int(0.1/dt), int(0.5/dt))
            
            try:
                p_max_ind = p_sum[:int(np.where(p_sum > p_sum.max()/2)[0][0] + 0.5/dt)].argmax()
                s_max_ind = get_s_index(s_sum)
            except:
                score[j, k] = 0
                mad_score[j, k] = 1000
                std_score[j, k] = 1000
            else:
                p_max = p_sum[p_max_ind]
                s_max = s_sum[s_max_ind]

                score[j, k] = p_max + s_max
                mad_score[j, k] = median_abs_deviation([p_max_ind, s_max_ind])
                std_score[j, k] = np.std([p_max_ind, s_max_ind])
    return i, score, mad_score, std_score

In [60]:
def calc_scores_cleaned(waveform_array, dt, dims, stations_eikonal_tables_vp, stations_eikonal_tables_vs):
    score = np.zeros((dims[0], dims[1], dims[2]))
    std_score = np.zeros((dims[0], dims[1], dims[2]))
    mad_score = np.zeros((dims[0], dims[1], dims[2]))
    
    indices = list(range(dims[0]))
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=81) as executor:
        for i, score_s, mad_score_s, std_score_s in executor.map(calc_scores_slice, repeat(waveform_array), repeat(dt), 
                                                                 repeat(dims), repeat(stations_eikonal_tables_vp), 
                                                                 repeat(stations_eikonal_tables_vs), indices):
            score[i, :, :] = score_s
            mad_score[i, :, :] = mad_score_s
            std_score[i, :, :] = std_score_s

    return score, std_score, mad_score

In [61]:
env_score_1, std_score_1, mad_score_1 = calc_scores_cleaned(waveform_array[201:402,:], dt, dims, stations_eikonal_tables_vp[:,:,:,201:], 
                                                  stations_eikonal_tables_vs[:,:,:,201:])

In [62]:
env_score_2, std_score_2, mad_score_2 = calc_scores_cleaned(waveform_array_2[:201,:], dt, dims, stations_eikonal_tables_vp[:,:,:,:201], 
                                                  stations_eikonal_tables_vs[:,:,:,:201])

In [73]:
def calc_scores_line_simple(waveform_array, dt, dims, j, k):
    global stations_eikonal_tables_vs_total
    score = np.zeros((dims[0]))

    for i in range(dims[0]):
        min_ind = int(np.ma.size(waveform_array, axis=1)/2) - \
                  int(np.round(np.min(np.abs(stations_eikonal_tables_vp[i, j, k, :]))/dt)) + int(0.25 / dt)
        max_ind = int(np.ma.size(waveform_array, axis=1)/2) - \
                  int(np.round(np.min(np.abs(stations_eikonal_tables_vs[i, j, k, :]))/dt)) + int(10 / dt)
        shifted_waveform_array_p = np.zeros(waveform_array.shape)
        shifted_waveform_array_s = np.zeros(waveform_array.shape)
        for l in range(np.ma.size(waveform_array, axis=0)):
            shifted_waveform_array_s[l, :] = \
                np.roll(waveform_array[l, :], -int(np.round(stations_eikonal_tables_vs_total[i, j, k, l] / dt)))

        shifted_waveform_array_s = shifted_waveform_array_s[:, min_ind:max_ind]

        s_sum = (shifted_waveform_array_s ** 2).sum(axis=0)

        s_sum = uniform_filter1d(s_sum, 30, mode='reflect')

        score[i] = s_sum.max()
    return j, score

In [98]:
def calc_scores_slice_multi(waveform_array, dt, dims, k):
    score = np.zeros((dims[0], dims[1]))

    indices = list(range(dims[1]))
        
    with concurrent.futures.ProcessPoolExecutor(max_workers=2) as executor:
        for i, score_s in executor.map(calc_scores_line_simple, repeat(waveform_array), repeat(dt), 
                                                                 repeat(dims), indices, repeat(k)):
            score[:, i] = score_s

    return k, score

In [113]:
def calc_time(waveform_array, dt, dims, stations_eikonal_tables_vs, i, j, k):
    shifted_waveform_array_s = np.zeros(waveform_array.shape)
    for l in range(np.ma.size(waveform_array, axis=0)):
        shifted_waveform_array_s[l, :] = \
            np.roll(waveform_array[l, :], -int(np.round(stations_eikonal_tables_vs[i, j, k, l] / dt)))

    shifted_waveform_array_s = shifted_waveform_array_s[:, :]

    s_sum = (shifted_waveform_array_s ** 2).sum(axis=0)

    s_sum = uniform_filter1d(s_sum, 30, mode='reflect')

    return (s_sum.argmax() - waveform_array.shape[1] / 2) * dt

In [96]:
def calc_scores_simple(waveform_array, dt, dims, depths_1):
    score = np.zeros((dims[0], dims[1], dims[2]))
    
    indices = depths_1
    
    with concurrent.futures.ProcessPoolExecutor(max_workers=34) as executor:
        for i, score_s in executor.map(calc_scores_slice_multi, repeat(waveform_array), repeat(dt), 
                                                                 repeat(dims), indices):
            score[:, :, i] = score_s

    return score

In [67]:
global stations_eikonal_tables_vs_total

In [99]:
score_2 = calc_scores_simple(waveform_array_2, dt, dims, depths_2)

In [90]:
with open("score_1.pk", "wb") as f:
    pickle.dump(score_1, f)

In [100]:
with open("score_2.pk", "wb") as f:
    pickle.dump(score_2, f)

In [243]:
t2 = calc_time(waveform_array, dt, dims, stations_eikonal_tables_vs_total, 65, 55, 58)

In [238]:
observed_seismograms.trim(starttime=obspy.UTCDateTime(0))

9659 Trace(s) in Stream:

DS.CI3108.lat=31.2723955707;long=35.2143378994;elev=0.0;bur=0.0;ort=[ 0.30671588 -0.95180112  0.        ].F | 1969-12-31T23:59:59.998999Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 6867 samples
...
(9657 other traces)
...
DS.CI3118.lat=31.2714454931;long=35.2146498967;elev=0.0;bur=0.0;ort=[ 0.3172723  -0.94833448  0.        ].F | 1969-12-31T23:59:59.998999Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 6867 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [68]:
def calc_probable_locations(scores):
    centre_of_mass_rep = score_to_probability(scores)
    geometric_median_est = estimate_geometric_median(centre_of_mass_rep)
    uncertainty, orientation = calc_confidence_ellipsoid(centre_of_mass_rep)
    probable_locations = [geometric_median_est, geometric_median_est + orientation[0] * uncertainty[0] / 2,
                          geometric_median_est - orientation[0] * uncertainty[0] / 2,
                          geometric_median_est + orientation[1] * uncertainty[1] / 2,
                          geometric_median_est - orientation[1] * uncertainty[1] / 2,
                          geometric_median_est + orientation[2] * uncertainty[2] / 2,
                          geometric_median_est - orientation[2] * uncertainty[2] / 2]
    return probable_locations

In [69]:
def score_to_probability(scores_array):
    prob_array = scores_array.copy()
    prob_array -= prob_array.min()
    wanted_half_val = prob_array.max() / 20
    prob_array -= prob_array.max()
    prob_array /= (wanted_half_val / np.log(2 * np.e - 1))
    prob_array[prob_array < -300] = -300
    prob_array = 1 / (1 + np.exp(-prob_array))
    prob_array /= prob_array.sum()

    return prob_array

In [70]:
def estimate_geometric_median(probability, eps=1e-5):
    points, weights = probability_to_point_list_prob(probability)
    y = np.mean(points, 0)

    while True:
        D = cdist(points, [y])
        nonzeros = (D != 0)[:, 0]
        zeros = (D == 0)[:, 0]
        Dinv = weights[nonzeros] / D[nonzeros]
        Dinvs = np.sum(Dinv)
        W = Dinv / Dinvs
        T = np.sum(W * points[nonzeros], 0)
        num_zeros = len(points) - np.sum(nonzeros)
        if num_zeros == 0:
            y1 = T
        elif num_zeros == len(points):
            return y
        else:
            R = (T - y) * Dinvs
            r = np.linalg.norm(R)
            eta = 0 if num_zeros == 0 else weights[zeros][0]
            rinv = 0 if r == 0 else eta / r
            y1 = max(0, 1 - rinv) * T + min(1, rinv) * y
        if euclidean(y, y1) < eps:
            return y1
        y = y1
    return y

In [71]:
def probability_to_point_list_prob(probability):
    xs = list(range(np.ma.size(probability, axis=0)))
    ys = list(range(np.ma.size(probability, axis=1)))
    zs = list(range(np.ma.size(probability, axis=2)))

    point_list = np.asarray(list(product(xs, ys, zs)))
    probability_1d = probability.ravel()
    probability_1d = probability_1d.reshape((np.ma.size(point_list, axis=0), 1))

    return point_list, probability_1d

In [72]:
def calc_confidence_ellipsoid(probability):
    points, weights = probability_to_point_list_prob(probability)
    weights_ext = np.squeeze(np.dstack([weights, weights, weights]))

    pca = WPCA().fit(points, weights=weights_ext)
    ellipsoid_orientation = pca.components_
    pca.explained_variance_[pca.explained_variance_ < 0] = 0
    half_uncertainties = np.sqrt(7.815 * pca.explained_variance_)

    return half_uncertainties, ellipsoid_orientation

In [123]:
prob_array_2 = score_2 - np.std(score_2) / np.std(std_score_2) * std_score_2
locations_2 = calc_probable_locations(prob_array_2)

In [75]:
aa = score_to_probability(-std_score_1-(-std_score_1).min())

In [77]:
bb = score_to_probability(-std_score_2-(-std_score_2).min())

In [163]:
cc = score_to_probability(-std_score_1-(-std_score_1).min())

In [144]:
def weighted_median(values, weights):
    i = np.argsort(values)
    c = np.cumsum(weights[i])
    return values[i[np.searchsorted(c, 0.5 * c[-1])]]

In [151]:
weighted_median(list(range(0, 81)), np.sum(np.sum(aa, axis=0), axis=0))

44

In [9]:
# observed_seismograms = SPECFEM3D_interface.read_observed_seismograms("test5/run0001", True)
with open("/DATA/eyal/specfem3d/PROJECTS/mtinv/test5/run0001/obseis.pk", "rb") as f:
    observed_seismograms = pickle.load(f)
observed_seismograms.filter("bandpass", freqmin=0.05, freqmax=10, zerophase=True)
observed_seismograms.normalize()

9659 Trace(s) in Stream:

DS.CI3108.lat=31.2723955707;long=35.2143378994;elev=0.0;bur=0.0;ort=[ 0.30671588 -0.95180112  0.        ].F | 1969-12-31T23:59:59.600000Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 7000 samples
...
(9657 other traces)
...
DS.CI3118.lat=31.2714454931;long=35.2146498967;elev=0.0;bur=0.0;ort=[ 0.3172723  -0.94833448  0.        ].F | 1969-12-31T23:59:59.600000Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 7000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [20]:
# observed_seismograms = SPECFEM3D_interface.read_observed_seismograms("test5/run0001", True)
with open("/DATA/eyal/specfem3d/PROJECTS/mtinv/test5/run0002/obseis.pk", "rb") as f:
    observed_seismograms_2 = pickle.load(f)
observed_seismograms_2.filter("bandpass", freqmin=0.05, freqmax=10, zerophase=True)
observed_seismograms_2.normalize()
observed_seismograms_2.sort(keys=["station"])

9659 Trace(s) in Stream:

DS.BI0.lat=31.21;long=35.3;elev=0.0;bur=0.0;ort=[ 0.  0.  1.].F | 1969-12-31T23:59:59.600000Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 7000 samples
...
(9657 other traces)
...
DS.CRY1800.lat=31.39;long=35.3;elev=0.0;bur=0.0;ort=[ 0.  1.  0.].F | 1969-12-31T23:59:59.600000Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 7000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [11]:
waveform_array = np.empty((len(observed_seismograms), observed_seismograms[0].stats.npts))

In [21]:
waveform_array_2 = np.empty((len(observed_seismograms_2), observed_seismograms_2[0].stats.npts))

In [10]:
observed_seismograms.sort(keys=["station"])

9659 Trace(s) in Stream:

DS.BI0.lat=31.21;long=35.3;elev=0.0;bur=0.0;ort=[ 0.  0.  1.].F | 1969-12-31T23:59:59.600000Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 7000 samples
...
(9657 other traces)
...
DS.CRY1800.lat=31.39;long=35.3;elev=0.0;bur=0.0;ort=[ 0.  1.  0.].F | 1969-12-31T23:59:59.600000Z - 1970-01-01T00:00:20.596937Z | 333.3 Hz, 7000 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]

In [119]:
observed_seismograms_trim = observed_seismograms.copy().trim(starttime=obspy.UTCDateTime(0))

In [120]:
waveform_array_trimmed = np.empty((len(observed_seismograms), observed_seismograms_trim[0].stats.npts))
for i in range(len(observed_seismograms)):
    waveform_array_trimmed[i] = observed_seismograms_trim[i].data
waveform_array_zeros = np.zeros(waveform_array_trimmed.shape)
waveform_array_trimmed = np.concatenate((waveform_array_zeros, waveform_array_trimmed), axis=1)

In [122]:
observed_seismograms_2_trim = observed_seismograms_2.copy().trim(starttime=obspy.UTCDateTime(0))
waveform_array_2_trimmed = np.empty((len(observed_seismograms), observed_seismograms_2_trim[0].stats.npts))
for i in range(len(observed_seismograms)):
    waveform_array_2_trimmed[i] = observed_seismograms_2_trim[i].data
waveform_array_zeros = np.zeros(waveform_array_2_trimmed.shape)
waveform_array_2_trimmed = np.concatenate((waveform_array_zeros, waveform_array_2_trimmed), axis=1)

In [12]:
for i in range(len(observed_seismograms)):
    waveform_array[i] = observed_seismograms[i].data

In [22]:
for i in range(len(observed_seismograms_2)):
    waveform_array_2[i] = observed_seismograms_2[i].data

In [13]:
waveform_array_zeros = np.zeros(waveform_array.shape)
waveform_array = np.concatenate((waveform_array_zeros, waveform_array), axis=1)

In [23]:
waveform_array_zeros = np.zeros(waveform_array_2.shape)
waveform_array_2 = np.concatenate((waveform_array_zeros, waveform_array_2), axis=1)

In [18]:
dt = observed_seismograms[0].stats.delta
dims = v_p.shape